# Practical Application III: Comparing Classifiers

**Overview**: In this practical application, your goal is to compare the performance of the classifiers we encountered in this section, namely K Nearest Neighbor, Logistic Regression, Decision Trees, and Support Vector Machines.  We will utilize a dataset related to marketing bank products over the telephone.  



### Getting Started

Our dataset comes from the UCI Machine Learning repository [link](https://archive.ics.uci.edu/ml/datasets/bank+marketing).  The data is from a Portugese banking institution and is a collection of the results of multiple marketing campaigns.  We will make use of the article accompanying the dataset [here](CRISP-DM-BANK.pdf) for more information on the data and features.



### Problem 1: Understanding the Data

To gain a better understanding of the data, please read the information provided in the UCI link above, and examine the **Materials and Methods** section of the paper.  How many marketing campaigns does this data represent?

In [1]:
print("17 campaigns")

17 campaigns


### Problem 2: Read in the Data

Use pandas to read in the dataset `bank-additional-full.csv` and assign to a meaningful variable name.

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('data/bank-additional-full.csv', sep = ';')

In [4]:
df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


### Problem 3: Understanding the Features


Examine the data description below, and determine if any of the features are missing values or need to be coerced to a different data type.


```
Input variables:
# bank client data:
1 - age (numeric)
2 - job : type of job (categorical: 'admin.','blue-collar','entrepreneur','housemaid','management','retired','self-employed','services','student','technician','unemployed','unknown')
3 - marital : marital status (categorical: 'divorced','married','single','unknown'; note: 'divorced' means divorced or widowed)
4 - education (categorical: 'basic.4y','basic.6y','basic.9y','high.school','illiterate','professional.course','university.degree','unknown')
5 - default: has credit in default? (categorical: 'no','yes','unknown')
6 - housing: has housing loan? (categorical: 'no','yes','unknown')
7 - loan: has personal loan? (categorical: 'no','yes','unknown')
# related with the last contact of the current campaign:
8 - contact: contact communication type (categorical: 'cellular','telephone')
9 - month: last contact month of year (categorical: 'jan', 'feb', 'mar', ..., 'nov', 'dec')
10 - day_of_week: last contact day of the week (categorical: 'mon','tue','wed','thu','fri')
11 - duration: last contact duration, in seconds (numeric). Important note: this attribute highly affects the output target (e.g., if duration=0 then y='no'). Yet, the duration is not known before a call is performed. Also, after the end of the call y is obviously known. Thus, this input should only be included for benchmark purposes and should be discarded if the intention is to have a realistic predictive model.
# other attributes:
12 - campaign: number of contacts performed during this campaign and for this client (numeric, includes last contact)
13 - pdays: number of days that passed by after the client was last contacted from a previous campaign (numeric; 999 means client was not previously contacted)
14 - previous: number of contacts performed before this campaign and for this client (numeric)
15 - poutcome: outcome of the previous marketing campaign (categorical: 'failure','nonexistent','success')
# social and economic context attributes
16 - emp.var.rate: employment variation rate - quarterly indicator (numeric)
17 - cons.price.idx: consumer price index - monthly indicator (numeric)
18 - cons.conf.idx: consumer confidence index - monthly indicator (numeric)
19 - euribor3m: euribor 3 month rate - daily indicator (numeric)
20 - nr.employed: number of employees - quarterly indicator (numeric)

Output variable (desired target):
21 - y - has the client subscribed a term deposit? (binary: 'yes','no')
```



In [5]:
# Handle missing values
missing_cols = ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'poutcome']
df[missing_cols] = df[missing_cols].replace('unknown', pd.NA)

# Convert data types
# Categorical variables
cat_cols = ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'day_of_week', 'poutcome', 'y']

# Numeric variables
num_cols = ['age', 'duration', 'campaign', 'pdays', 'previous', 'emp.var.rate', 'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed']
df[num_cols] = df[num_cols].astype('float64')  # Convert to float to handle potential NaN values

# Fill missing categorical values with 'unknown'
df[cat_cols] = df[cat_cols].fillna('unknown')

# Verify if all missing categorical values have been filled
print(df[cat_cols].isnull().sum())

# Convert 'Y' column to boolean value
df['y'] = df['y'].map({'yes': True, 'no': False})

# Display the data types and check for missing values
print(df.info())
print(df.isnull().sum())


job            0
marital        0
education      0
default        0
housing        0
loan           0
contact        0
month          0
day_of_week    0
poutcome       0
y              0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             41188 non-null  float64
 1   job             41188 non-null  object 
 2   marital         41188 non-null  object 
 3   education       41188 non-null  object 
 4   default         41188 non-null  object 
 5   housing         41188 non-null  object 
 6   loan            41188 non-null  object 
 7   contact         41188 non-null  object 
 8   month           41188 non-null  object 
 9   day_of_week     41188 non-null  object 
 10  duration        41188 non-null  float64
 11  campaign        41188 non-null  float64
 12  pdays           41188 non-null  float64
 13  previous   

In [6]:
# Convert categorical variables to one-hot encoding
df_encoded = pd.get_dummies(df, columns=cat_cols)

# Display the encoded DataFrame
print(df_encoded.head())


    age  duration  campaign  pdays  previous  emp.var.rate  cons.price.idx  \
0  56.0     261.0       1.0  999.0       0.0           1.1          93.994   
1  57.0     149.0       1.0  999.0       0.0           1.1          93.994   
2  37.0     226.0       1.0  999.0       0.0           1.1          93.994   
3  40.0     151.0       1.0  999.0       0.0           1.1          93.994   
4  56.0     307.0       1.0  999.0       0.0           1.1          93.994   

   cons.conf.idx  euribor3m  nr.employed  ...  day_of_week_fri  \
0          -36.4      4.857       5191.0  ...            False   
1          -36.4      4.857       5191.0  ...            False   
2          -36.4      4.857       5191.0  ...            False   
3          -36.4      4.857       5191.0  ...            False   
4          -36.4      4.857       5191.0  ...            False   

   day_of_week_mon  day_of_week_thu  day_of_week_tue  day_of_week_wed  \
0             True            False            False         

### Problem 4: Understanding the Task

After examining the description and data, your goal now is to clearly state the *Business Objective* of the task.  State the objective below.

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             41188 non-null  float64
 1   job             41188 non-null  object 
 2   marital         41188 non-null  object 
 3   education       41188 non-null  object 
 4   default         41188 non-null  object 
 5   housing         41188 non-null  object 
 6   loan            41188 non-null  object 
 7   contact         41188 non-null  object 
 8   month           41188 non-null  object 
 9   day_of_week     41188 non-null  object 
 10  duration        41188 non-null  float64
 11  campaign        41188 non-null  float64
 12  pdays           41188 non-null  float64
 13  previous        41188 non-null  float64
 14  poutcome        41188 non-null  object 
 15  emp.var.rate    41188 non-null  float64
 16  cons.price.idx  41188 non-null  float64
 17  cons.conf.idx   41188 non-null 

In [8]:
print('The goal is to predict if the client will subscribe a term deposit (which is variable y).')

The goal is to predict if the client will subscribe a term deposit (which is variable y).


### Problem 5: Engineering Features

Now that you understand your business objective, we will build a basic model to get started.  Before we can do this, we must work to encode the data.  Using just the bank information features (columns 1 - 7), prepare the features and target column for modeling with appropriate encoding and transformations.

In [9]:
# Select the first seven columns with target value
df_first_seven = df.iloc[:, [0, 1, 2, 3, 4, 5, 6, -1]]

print(df_first_seven)

# Convert categorical variables to one-hot encoding for the selected columns
df_seven_encoded = pd.get_dummies(df_first_seven)

# Display the final encoded DataFrame
print(df_seven_encoded.info())

# Display the encoded DataFrame
print(df_seven_encoded.head())

# Join the target variable 'Y' with the encoded DataFrame
# df_encoded_with_y = pd.concat([df_seven_encoded, df['y']], axis=1)

# # Display the final encoded DataFrame
# print(df_encoded_with_y.info())

# # Display the encoded DataFrame
# print(df_encoded_with_y.head())


        age          job  marital            education  default housing loan  \
0      56.0    housemaid  married             basic.4y       no      no   no   
1      57.0     services  married          high.school  unknown      no   no   
2      37.0     services  married          high.school       no     yes   no   
3      40.0       admin.  married             basic.6y       no      no   no   
4      56.0     services  married          high.school       no      no  yes   
...     ...          ...      ...                  ...      ...     ...  ...   
41183  73.0      retired  married  professional.course       no     yes   no   
41184  46.0  blue-collar  married  professional.course       no      no   no   
41185  56.0      retired  married    university.degree       no     yes   no   
41186  44.0   technician  married  professional.course       no      no   no   
41187  74.0      retired  married  professional.course       no     yes   no   

           y  
0      False  
1      Fa

### Problem 6: Train/Test Split

With your data prepared, split it into a train and test set.

In [10]:
from sklearn.model_selection import train_test_split

# Split the data into features (X) and target variable (y)
X = df_seven_encoded.drop('y', axis=1)  # Features
y = df_seven_encoded['y']  # Target variable

# Split the data into train and test sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Display the shapes of the train and test sets
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)


X_train shape: (32950, 34)
y_train shape: (32950,)
X_test shape: (8238, 34)
y_test shape: (8238,)


### Problem 7: A Baseline Model

Before we build our first model, we want to establish a baseline.  What is the baseline performance that our classifier should aim to beat?

In [11]:

# Determine the majority class ('yes' or 'no')
majority_class = df['y'].value_counts().idxmax()

# Count the number of samples belonging to the majority class
majority_class_count = df['y'].value_counts()[majority_class]

# Calculate the percentage of samples belonging to the majority class
total_samples = df.shape[0]
baseline_accuracy = majority_class_count / total_samples * 100

print("Baseline accuracy: {:.2f}%".format(baseline_accuracy))


Baseline accuracy: 88.73%


### Problem 8: A Simple Model

Use Logistic Regression to build a basic model on your data.  

In [12]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, confusion_matrix

# Build a logistic regression model
model = LogisticRegression()

# Fit the model on the training data
model.fit(X_train, y_train)

# Predict on the test data
y_pred = model.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Print classification report
print("Classification Report:")
print(classification_report(y_test, y_pred))

# Print confusion matrix
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))


Accuracy: 0.8865015780529255
Classification Report:
              precision    recall  f1-score   support

       False       0.89      1.00      0.94      7303
        True       0.00      0.00      0.00       935

    accuracy                           0.89      8238
   macro avg       0.44      0.50      0.47      8238
weighted avg       0.79      0.89      0.83      8238

Confusion Matrix:
[[7303    0]
 [ 935    0]]


/Users/mrburstein/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/mrburstein/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/mrburstein/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score

### Problem 9: Score the Model

What is the accuracy of your model?

In [13]:
print("Accuracy:", accuracy)


Accuracy: 0.8865015780529255


### Problem 10: Model Comparisons

Now, we aim to compare the performance of the Logistic Regression model to our KNN algorithm, Decision Tree, and SVM models.  Using the default settings for each of the models, fit and score each.  Also, be sure to compare the fit time of each of the models.  Present your findings in a `DataFrame` similar to that below:

| Model | Train Time | Train Accuracy | Test Accuracy |
| ----- | ---------- | -------------  | -----------   |
|     |    |.     |.     |

In [14]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import time
import pandas as pd

import numpy as np

# Convert X_train and X_test to NumPy arrays if they are not already
X_train = np.array(X_train)
X_test = np.array(X_test)

# Define models
models = {
    'Logistic Regression': LogisticRegression(),
    'KNN': KNeighborsClassifier(),
    'Decision Tree': DecisionTreeClassifier(),
    'SVM': SVC()
}

# Fit and evaluate models
results = [
    {
        'Model': model_name,
        'Train Time': "{:.9f} s".format(start_time := time.time(), time.time() - start_time)[1],
        'Train Accuracy': (model.fit(X_train, y_train).score(X_train, y_train)),
        'Test Accuracy': model.score(X_test, y_test)
    }
    for model_name, model in models.items()
]

# Create DataFrame from results
results_df = pd.DataFrame(results)

# Print results
print(results_df)


/Users/mrburstein/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


                 Model Train Time  Train Accuracy  Test Accuracy
0  Logistic Regression          7        0.887557       0.886502
1                  KNN          7        0.890015       0.873877
2        Decision Tree          7        0.916601       0.862102
3                  SVM          7        0.887557       0.886502


### Problem 11: Improving the Model

Now that we have some basic models on the board, we want to try to improve these.  Below, we list a few things to explore in this pursuit.

- More feature engineering and exploration.  For example, should we keep the gender feature?  Why or why not?
- Hyperparameter tuning and grid search.  All of our models have additional hyperparameters to tune and explore.  For example the number of neighbors in KNN or the maximum depth of a Decision Tree.  
- Adjust your performance metric

In [15]:
# 1. More feature engineering and exploration
# instead of keeping the first seven columns we will include features i believe would be crucial to make a more accurate decision 

# Select the first seven columns with target value
df_final = df.iloc[:, [0, 1, 2, 3, 4, 5, 6, 11, 12, 13, 14, 15, -1]]

print(df_final)

# Convert categorical variables to one-hot encoding for the selected columns
df_final_encoded = pd.get_dummies(df_final)

# Display the final encoded DataFrame
print(df_final_encoded.info())



        age          job  marital            education  default housing loan  \
0      56.0    housemaid  married             basic.4y       no      no   no   
1      57.0     services  married          high.school  unknown      no   no   
2      37.0     services  married          high.school       no     yes   no   
3      40.0       admin.  married             basic.6y       no      no   no   
4      56.0     services  married          high.school       no      no  yes   
...     ...          ...      ...                  ...      ...     ...  ...   
41183  73.0      retired  married  professional.course       no     yes   no   
41184  46.0  blue-collar  married  professional.course       no      no   no   
41185  56.0      retired  married    university.degree       no     yes   no   
41186  44.0   technician  married  professional.course       no      no   no   
41187  74.0      retired  married  professional.course       no     yes   no   

       campaign  pdays  previous     po

In [16]:
# Split the data into features (X) and target variable (y)
X = df_final_encoded.drop('y', axis=1)  # Features
y = df_final_encoded['y']  # Target variable

# Split the data into train and test sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Display the shapes of the train and test sets
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)

X_train shape: (32950, 41)
y_train shape: (32950,)
X_test shape: (8238, 41)
y_test shape: (8238,)


In [17]:
from sklearn.model_selection import GridSearchCV


# 2. Hyperparameter tuning and grid search

# Convert X_train and X_test to NumPy arrays if they are not already
X_train = np.array(X_train)
X_test = np.array(X_test)

# Define models
models = {
    'Logistic Regression': LogisticRegression(),
    'KNN': KNeighborsClassifier(),
    'Decision Tree': DecisionTreeClassifier(),
    #'SVM': SVC()
}

# Define hyperparameters to tune for each model
param_grid = {
    'KNN': {'n_neighbors': [3, 5, 7, 9]},
    'Decision Tree': {'max_depth': [None, 5, 10, 20]},
    #'SVM': {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf']},
    'Logistic Regression': {'C': [0.1, 1, 10], 'penalty': ['l1', 'l2']}
}
# Iterate over models
for model_name, model in models.items():
    # Perform grid search with cross-validation
    grid_search = GridSearchCV(model, param_grid[model_name], cv=5, scoring='accuracy')
    grid_search.fit(X_train, y_train)
    
    # Print best hyperparameters
    print(f"Best hyperparameters for {model_name}: {grid_search.best_params_}")
    
    # Evaluate model performance on test set
    y_pred = grid_search.predict(X_test)
    print(f"Classification report for {model_name}:")
    print(classification_report(y_test, y_pred))

    # Print training time with more digits
    train_time = grid_search.cv_results_['mean_fit_time'][grid_search.best_index_]
    print(f"Train Time: {train_time:.9f} s")


/Users/mrburstein/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/mrburstein/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#l

Best hyperparameters for Logistic Regression: {'C': 1, 'penalty': 'l2'}
Classification report for Logistic Regression:
              precision    recall  f1-score   support

       False       0.91      0.98      0.94      7303
        True       0.63      0.20      0.30       935

    accuracy                           0.90      8238
   macro avg       0.77      0.59      0.62      8238
weighted avg       0.87      0.90      0.87      8238

Train Time: 0.408423281 s
Best hyperparameters for KNN: {'n_neighbors': 9}
Classification report for KNN:
              precision    recall  f1-score   support

       False       0.91      0.98      0.94      7303
        True       0.52      0.20      0.29       935

    accuracy                           0.89      8238
   macro avg       0.71      0.59      0.62      8238
weighted avg       0.86      0.89      0.87      8238

Train Time: 0.023102045 s
Best hyperparameters for Decision Tree: {'max_depth': 5}
Classification report for Decision Tre

In [18]:
import pandas as pd
from sklearn.metrics import precision_score, recall_score, f1_score

# Define the performance metrics to evaluate
metrics = {'Precision': precision_score, 'Recall': recall_score, 'F1-score': f1_score}

# Initialize empty list to store results
results = []

# Iterate over models
for model_name, model in models.items():
    # Fit the model
    model.fit(X_train, y_train)
    
    # Calculate train accuracy
    train_accuracy = model.score(X_train, y_train)
    
    # Calculate test accuracy
    test_accuracy = model.score(X_test, y_test)
    
    # Evaluate the model using different metrics
    metric_scores = {}
    for metric_name, metric_func in metrics.items():
        # Calculate the metric score
        metric_score = metric_func(y_test, model.predict(X_test))
        metric_scores[metric_name] = metric_score
    
    # Append results to list
    results.append({
        'Model': model_name,
        'Train Accuracy': train_accuracy,
        'Test Accuracy': test_accuracy,
        **metric_scores  # Add additional metrics to the results
    })

# Create DataFrame from the results list
results_df = pd.DataFrame(results)

# Print the results DataFrame
print(results_df)


/Users/mrburstein/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


                 Model  Train Accuracy  Test Accuracy  Precision    Recall  \
0  Logistic Regression        0.898877       0.895970   0.630872  0.201070   
1                  KNN        0.909894       0.882617   0.464758  0.225668   
2        Decision Tree        0.979757       0.839888   0.297040  0.300535   

   F1-score  
0  0.304947  
1  0.303816  
2  0.298777  


##### Questions